In [4]:
############
#INSTALL PySyft
##########

!git clone https://github.com/OpenMined/PySyft.git
!cd PySyft/
!pip install -r PySyft/pip-dep/requirements.txt
!pip install -r PySyft/pip-dep/requirements_udacity.txt
!python PySyft/setup.py install

Cloning into 'PySyft'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 40287 (delta 47), reused 26 (delta 9), pack-reused 40186
Receiving objects: 100% (40287/40287), 38.72 MiB | 21.05 MiB/s, done.
Resolving deltas: 100% (27444/27444), done.
     |████████████████████████████████| 1.8MB 7.6MB/s 
     |████████████████████████████████| 2.1MB 31.1MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 204kB 39.5MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 1.3MB 43.0MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for phe: filename=phe-1.4.0-py2.py3-none-any.whl size=37362 sha256=ee94f0c0815a09600a3cf17591602adc4ba769bce0c7fc74d55e71c20d1915e1
  Stored in 

     |████████████████████████████████| 2.7MB 5.0MB/s 
     |████████████████████████████████| 276kB 41.9MB/s 
     |████████████████████████████████| 110.5MB 107kB/s 
     |████████████████████████████████| 3.8MB 26.2MB/s 
     |████████████████████████████████| 512kB 38.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=b5a8bac6ce72c357374cdf06106afda6163a2d9a7dfc66591d11ca747ce3e35f
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e123eff4686c29991b39b79b361312c090dad3bccb5d92b0e06bc284ed7e343f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built pyyaml gast
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: tensorboard 2.2.0
   

Before Running the next step make sure to "Restart Runtime"

In [0]:
# Run this cell to add PySyft path 
import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import copy

Import PySyft and create hook to use torch libraries

In [3]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


Create Four agents. Each of them will be used to train on the data independently (sequentially though!)

In [0]:
Agent_1 = sy.VirtualWorker(hook, id="Agent_1")
Agent_2 = sy.VirtualWorker(hook, id="Agent_2")
Agent_3 = sy.VirtualWorker(hook, id="Agent_3")
Agent_4 = sy.VirtualWorker(hook, id="Agent_4")

Arguments to be used for training. I have taken this directly from the PySyft tutorial

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 1
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [6]:
# Download dataset

mnist_full_train_dataset = datasets.MNIST('../data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


Split the dataset into three parts. This is done to mimic training on the data at three different "time stamps". The first dataset will be used to train the model with no malicious data. We will include some malicious data in datasets 2 and 3 and make sure the model obtained from dataset 1 is not updated by a malicious agent's updated delta

In [0]:

ds_t1,ds_t2,ds_t3 = torch.utils.data.random_split(mnist_full_train_dataset,(20000,20000,20000))

# TO DO
How to manipulate dataset to include malicious updates???

In [9]:
temp = ds_t2
print(ds_t2[10][1])
temp[10][1] = 4
print(temp[10][1])

9


TypeError: ignored

Create three "federated" datastes. Each corresponding to the dataset created above. This is done to use "PySyft" infrastructure to mimic training model using four different slaves (or clients or agents)

In [0]:
federated_train_ds_t1 = sy.FederatedDataLoader( ds_t1.federate((Agent_1,Agent_2,Agent_3,Agent_4)),batch_size=64,shuffle=True, **kwargs)
federated_train_ds_t2 = sy.FederatedDataLoader( ds_t2.federate((Agent_1,Agent_2,Agent_3,Agent_4)),batch_size=64,shuffle=True, **kwargs)
federated_train_ds_t3 = sy.FederatedDataLoader( ds_t3.federate((Agent_1,Agent_2,Agent_3,Agent_4)),batch_size=64,shuffle=True, **kwargs)

In [0]:
# This is the dataset that will be used repeateadly to test model performance
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

CNN model. I have taken this directly from the PySyft tutorial. We can potentialy look at changig this but I have not spent time on this yet.

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

This is the train function. This is sort of a hack solution to train the model on the 1st dataset using PySyft's federated training

In [0]:
def train(args, model, device, federated_train_loader, optimizer):
    
    model.train()
        
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        cal_grad_bkpropgt(data,target,batch_idx,federated_train_loader,model,device)

Test fucniton to test model performance

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Helper function to calculate gradient. This will be used on the 1st dataset when training on non-malicious data to get a basline model

In [0]:
def cal_grad_bkpropgt(data,target,batch_idx,federated_train_loader,model,device):
  model.send(data.location) # <-- NEW: send the model to the right location
  data, target = data.to(device), target.to(device)
  optimizer.zero_grad()
  output = model(data)
  loss = F.nll_loss(output, target)
  loss.backward()
  optimizer.step()
  model.get() # <-- NEW: get the model back
  if batch_idx % args.log_interval == 0:
      loss = loss.get() # <-- NEW: get the loss back
      print('Agent: {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          data.location,1, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
          100. * batch_idx / len(federated_train_loader), loss.item()))
  
  #return model

Helper function to get delta's from each Agent when training on datasets 2 and 3

In [0]:
def cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device):
  #org_model_dict = model.state_dict()
  #print('Starting training on batch', batch_idx)
  model.send(data.location) # <-- NEW: send the model to the right location
  data, target = data.to(device), target.to(device)
  optimizer.zero_grad()
  output = model(data)
  loss = F.nll_loss(output, target)
  loss.backward()
  optimizer.step()
  model.get() # <-- NEW: get the model back
      
  #return model

Train model on dataset 1 (remember dataset 1 corresponds to training at timestamp 1)

In [26]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

train(args, model, device, federated_train_ds_t1, optimizer)
test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



Agent: <VirtualWorker id:Agent_1 #objects:11> Train Epoch: 1 [0/20032 (0%)]	Loss: 2.289706
Agent: <VirtualWorker id:Agent_1 #objects:11> Train Epoch: 1 [1920/20032 (10%)]	Loss: 2.167943
Agent: <VirtualWorker id:Agent_1 #objects:11> Train Epoch: 1 [3840/20032 (19%)]	Loss: 1.859887
Agent: <VirtualWorker id:Agent_2 #objects:11> Train Epoch: 1 [5760/20032 (29%)]	Loss: 1.365499
Agent: <VirtualWorker id:Agent_2 #objects:11> Train Epoch: 1 [7680/20032 (38%)]	Loss: 0.804456
Agent: <VirtualWorker id:Agent_2 #objects:11> Train Epoch: 1 [9600/20032 (48%)]	Loss: 0.464902
Agent: <VirtualWorker id:Agent_3 #objects:11> Train Epoch: 1 [11520/20032 (58%)]	Loss: 0.473106
Agent: <VirtualWorker id:Agent_3 #objects:11> Train Epoch: 1 [13440/20032 (67%)]	Loss: 0.495449
Agent: <VirtualWorker id:Agent_4 #objects:15> Train Epoch: 1 [15360/20032 (77%)]	Loss: 0.519198
Agent: <VirtualWorker id:Agent_4 #objects:15> Train Epoch: 1 [17280/20032 (86%)]	Loss: 0.546559
Agent: <VirtualWorker id:Agent_4 #objects:15> Trai

Function to train on datasets 2 and 3 corresponding to time stamps t2 and t3.

In [0]:
def train_subsequent_trainings(args, model,Current_model, device, federated_train_loader, optimizer):

  #model.train()
     
  for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
      
      if data.location == Agent_1:
        #print('Working on Agent 1')
        cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device)
          

      
      elif data.location == Agent_2:
        
        #print('Working on Agent 2')
        delta_Agent_1 = {name: Current_model.state_dict()[name] - model.state_dict()[name] for name in Current_model.state_dict() if name in model.state_dict()}
        model.load_state_dict(Current_model.state_dict())
        cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device)
        
      
      elif data.location == Agent_3:
        #print('Working on Agent 3')
        delta_Agent_2 = {name: Current_model.state_dict()[name] - model.state_dict()[name] for name in Current_model.state_dict() if name in model.state_dict()}
        model.load_state_dict(Current_model.state_dict())
        cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device)
        
      
      elif data.location == Agent_4:
        #print('Working on Agent 4')
        delta_Agent_3 = {name: Current_model.state_dict()[name] - model.state_dict()[name] for name in Current_model.state_dict() if name in model.state_dict()}
        model.load_state_dict(Current_model.state_dict())
        cal_grad_bkpropgt_return_delta(data,target,batch_idx,federated_train_loader,model,device)

      else:
        pass
  
  delta_Agent_4 = {name: Current_model.state_dict()[name] - model.state_dict()[name] for name in Current_model.state_dict() if name in model.state_dict()}

  
  return delta_Agent_1, delta_Agent_2, delta_Agent_3, delta_Agent_4    


In [28]:
Current_model = Net().to(device)
Current_model.load_state_dict(model.state_dict())

#train_subsequent_trainings(args, model, device, federated_train_ds_t2, optimizer)

<All keys matched successfully>

In [29]:
%%time

# Get deltas from each agent for dataset 2 (corresponding to tiemstamp 2)

delta_Agent_1, delta_Agent_2, delta_Agent_3, delta_Agent_4 = train_subsequent_trainings(args, model,Current_model, device, federated_train_ds_t2, optimizer)

#test(args, model, device, test_loader)




CPU times: user 44.8 s, sys: 2.81 s, total: 47.6 s
Wall time: 47.8 s


In [32]:
# Compare performance by adding delta from Agent 1 vs average delta from Agent2, 3, and 4

# calculate average delta for Agent 2, 3, and 4
for name,param in delta_Agent_2.items():
  average_delta_Agent2_3_4 = {name: (delta_Agent_2[name]+ delta_Agent_3[name]+delta_Agent_4[name])/3 for name in delta_Agent_2 if 
                              name in delta_Agent_3 and name in delta_Agent_4}

# get new model dict by adding delta from Agent 1                              
for name,param in delta_Agent_1.items():
  #print(name,param)
  mod_state_dict_Agent1 = {name: Current_model.state_dict()[name] - delta_Agent_1[name] for name in delta_Agent_1}

Test_model = Net().to(device)
Test_model.load_state_dict(mod_state_dict_Agent1)


# calculate performance by adding delta from Agent 1
test(args, Test_model, device, test_loader)


Test set: Average loss: 0.4283, Accuracy: 8728/10000 (87%)



In [33]:
for name,param in delta_Agent_2.items():
  mod_state_dict_Agent2_3_4 = {name: Current_model.state_dict()[name] - average_delta_Agent2_3_4[name] for name in delta_Agent_2}

Test_model.load_state_dict(mod_state_dict_Agent2_3_4)
# calculate performance by adding delta from Agent 1
test(args, Test_model, device, test_loader)




Test set: Average loss: 0.5877, Accuracy: 8278/10000 (83%)



In [40]:
test(args, model, device, test_loader)


Test set: Average loss: 0.3243, Accuracy: 9056/10000 (91%)

